In [120]:
import os
import numpy as np
import pandas as pd
import io

In [121]:
pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

In [122]:
# First, I want to import all my .xls files at once

def loadXlsFiles(folder_paths):
    dataframes = []
    for folder_path in folder_paths:
        files = os.listdir(folder_path)
        excel_files = [f for f in files if f.endswith('.xls')]
# Loop through the list of Excel files
        for file in excel_files:
            file_path = os.path.join(folder_path, file)  # Full path to the file
            df = pd.read_excel(file_path)  # Read the Excel file into a DataFrame
            dataframes.append(df)  # Append the DataFrame to the list
    return dataframes
    

folder_paths =['../My Data Science Projects/BW Reservations/2021-2022', 
               '../My Data Science Projects/BW Reservations/2022-2023',
              '../My Data Science Projects/BW Reservations/2023-2024',
              '../My Data Science Projects/BW Reservations/2024-2025',]


dataframes = loadXlsFiles(folder_paths)

In [123]:
# Remove operator summary table and its entries that are uninentionally blended with guest data during pd.read_excel
# As well as observations containing entirely null column values


for i in range(len(dataframes)):
    # Remove rows where 'Guest Name' column equals 'Operator Summary'
    dataframes[i] = dataframes[i][dataframes[i]['Guest Name'] != 'Operator Summary']

for dataframe in dataframes:
    dataframe = dataframe.dropna(how='all',inplace=True)
    
# For loop to reliably remove 'operator summary' table data from each data frame 
for i in range(len(dataframes)):
    # Find the index of the first occurrence of 'Admin' and 'Totals'
    start_index = dataframes[i][dataframes[i]['Guest Name'] == 'Admin'].index.min()
    end_index = dataframes[i][dataframes[i]['Guest Name'] == 'Totals'].index.min()

    # Drop rows from start_index to end_index, inclusive
    if pd.notna(start_index) and pd.notna(end_index) and start_index <= end_index:
        dataframes[i].drop(dataframes[i].loc[start_index:end_index].index, inplace=True)


bookings_21_24_df = pd.concat(dataframes)
bookings_21_24_df.reset_index(drop=True, inplace=True)

### Since this is real world data, I am going to hash the names of the guest via SHA-256 hash algorithm using hashlib. This is simple and the same input will always yield the same hash output so repeat Guest Name values are not lost during in the encryption process.

In [124]:
import hashlib


# Function to hash text using SHA-256
def hash_name(name):
    # Convert the name to a byte string, then hash it
    hash_object = hashlib.sha256(name.encode())
    # Return the hexadecimal representation of the digest
    return hash_object.hexdigest()

# Hashing names in the DataFrame
bookings_21_24_df['Guest Name'] = bookings_21_24_df['Guest Name'].apply(hash_name)

bookings_21_24_df

,Guest Name,Status,Arrive,Nts,Company,Made,Type,Rate,Disc,Override,PKG,Grp ID,Clerk,Conf #,Unnamed: 14
0,a6adf0970ec595255cb5417a5dfe6ce0f0c4608ed93dc0...,CXL,01/01/21,1,FRATERNAL OR,12/31/20,SK,119,NaN,0,NaN,NaN,ALeRoy,84406,NaN
1,a9d6016b3827799118c1f1feea23d00068b8cf79d9b40e...,OUT,01/01/21,1,NaN,12/31/20,SQQ,116.1,AAA,0,NaN,NaN,Tdahmer,84416,NaN
2,70907cf1d1f5659b43c5fb616417a0b87975cb93e0abe9...,OUT,01/01/21,1,NaN,01/01/21,SK,119,NaN,0,NaN,NaN,ALeRoy,84422,NaN
3,c098f7f6aab9e1288456ee87e57a19c100d7d6f861e0ff...,CXL,01/01/21,1,NaN,01/01/21,SQQ,109.65,MR,0,NaN,NaN,Lobrigo,84431,NaN
4,f112a154cdc31bfd940fec55423f8ed38e4018642fade4...,OUT,01/01/21,2,NaN,12/28/20,SQQ,98.1,BW,98.1,NaN,NaN,Admin,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51722,05b6d87c80678981de63ef2b3472c6b72da1b4b9ca703f...,GTD,05/07/24,1,NaN,02/13/24,SK,119,BC9,93.18,NaN,NaN,Admin,625258271-01,NaN
51723,8da1d8cff528fbc332a062146526ad2e522097a0734b7d...,CXL,05/07/24,1,(EPS) EXPEDI,04/16/24,SKA,95.46,EC9,95.4,NaN,NaN,Admin,860986212-01,NaN
51724,a2fc3ad7736a52b9aaa7ea983d15b30ddfd0d4ab1aa409...,GTD,05/07/24,2,NaN,04/19/24,SQQ,120.84,ECR,121.64,NaN,NaN,Admin,209536113-01,NaN
51725,4745091e4e1a796c1375584cabee8d66ae606655730d74...,GTD,05/07/24,2,NaN,04/17/24,SQQ,120.93,BW,120.93,NaN,NaN,Admin,776126515-01,NaN


In [125]:
bookings_21_24_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51727 entries, 0 to 51726
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Guest Name   51727 non-null  object
 1   Status       51727 non-null  object
 2   Arrive       51727 non-null  object
 3   Nts          51727 non-null  object
 4   Company      7158 non-null   object
 5   Made         51727 non-null  object
 6   Type         51727 non-null  object
 7   Rate         51727 non-null  object
 8   Disc         35487 non-null  object
 9   Override     51727 non-null  object
 10  PKG          0 non-null      object
 11  Grp ID       2902 non-null   object
 12  Clerk        51727 non-null  object
 13  Conf #       34991 non-null  object
 14  Unnamed: 14  0 non-null      object
dtypes: object(15)
memory usage: 5.9+ MB


In [126]:
bookings_21_24_df['Type'].value_counts()

SK        27515
SQQ       16757
JRQ        2392
SDD        1714
EQ         1143
SKA         878
JRD         671
PJ          656
ZZFA          1
Name: Type, dtype: int64

### The data has many null values, especially in ' PKG', 'Grp ID', and 'Unnamed: 14'.
### Additionally  'Guest Name' has 1 null value and there are other columns that are missing only a few values.
### I am considering simply dropping the observations that contain null values from the columns with a high percentage of non-null values.

In [127]:
# Create a copy of the raw data
# Create a series countaining non-null percentages to iterate over
data = bookings_21_24_df.copy()
non_null_pct = (data.count() / data.shape[0] * 100)
non_null_pct

Guest Name     100.000000
Status         100.000000
Arrive         100.000000
Nts            100.000000
Company         13.838034
Made           100.000000
Type           100.000000
Rate           100.000000
Disc            68.604404
Override       100.000000
PKG              0.000000
Grp ID           5.610223
Clerk          100.000000
Conf #          67.645524
Unnamed: 14      0.000000
dtype: float64

In [128]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51727 entries, 0 to 51726
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Guest Name   51727 non-null  object
 1   Status       51727 non-null  object
 2   Arrive       51727 non-null  object
 3   Nts          51727 non-null  object
 4   Company      7158 non-null   object
 5   Made         51727 non-null  object
 6   Type         51727 non-null  object
 7   Rate         51727 non-null  object
 8   Disc         35487 non-null  object
 9   Override     51727 non-null  object
 10  PKG          0 non-null      object
 11  Grp ID       2902 non-null   object
 12  Clerk        51727 non-null  object
 13  Conf #       34991 non-null  object
 14  Unnamed: 14  0 non-null      object
dtypes: object(15)
memory usage: 5.9+ MB


In [129]:
# for loop to drop rows containing non-null values for each column with a non null count percentage of 81.1% or higher.
for column in non_null_pct.index:
    if non_null_pct[column] > 81.1:
        data.dropna(subset=column, inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51727 entries, 0 to 51726
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Guest Name   51727 non-null  object
 1   Status       51727 non-null  object
 2   Arrive       51727 non-null  object
 3   Nts          51727 non-null  object
 4   Company      7158 non-null   object
 5   Made         51727 non-null  object
 6   Type         51727 non-null  object
 7   Rate         51727 non-null  object
 8   Disc         35487 non-null  object
 9   Override     51727 non-null  object
 10  PKG          0 non-null      object
 11  Grp ID       2902 non-null   object
 12  Clerk        51727 non-null  object
 13  Conf #       34991 non-null  object
 14  Unnamed: 14  0 non-null      object
dtypes: object(15)
memory usage: 5.9+ MB


### As for the other columns, I will inspect them further before omitting them from the dataset

In [130]:
# list comprehension to create a list of high null-count columns

high_nullcount_columns = [column for column in non_null_pct.index if non_null_pct[column] < 81.1]
high_nullcount_columns

['Company', 'Disc', 'PKG', 'Grp ID', 'Conf #', 'Unnamed: 14']

In [131]:
# Pass those columns into our original data to filter them out
high_nullcount_df = data[high_nullcount_columns]
high_nullcount_df

,Company,Disc,PKG,Grp ID,Conf #,Unnamed: 14
0,FRATERNAL OR,NaN,NaN,NaN,84406,NaN
1,NaN,AAA,NaN,NaN,84416,NaN
2,NaN,NaN,NaN,NaN,84422,NaN
3,NaN,MR,NaN,NaN,84431,NaN
4,NaN,BW,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
51722,NaN,BC9,NaN,NaN,625258271-01,NaN
51723,(EPS) EXPEDI,EC9,NaN,NaN,860986212-01,NaN
51724,NaN,ECR,NaN,NaN,209536113-01,NaN
51725,NaN,BW,NaN,NaN,776126515-01,NaN


### Upon inspection of this dataframe, many of the null values are to be expected given the category of their respective columns.
#### For example, a value of NaN in the 'Company' column indicates that the customer did not book through a third party Company. Therefore, it is highly intuitive to simply replace the null values in the Company column with 0's

#### Similarly, in the 'Disc' (Discount) column, we see that most customers booked without a discount. It also makes sense to replace these null values with 0's to indicate that the customer did not receive a discount on that particular booking observation.


#### For the 'PKG',  'Grp ID' and 'Unamed: 14' columns, almost all the values are null. The values for these columns appear exclusively in the last several rows of the dataframe. That is because of the way the company exports the xlsl file we initally read from. These columns are only asscoiated with observations where 'Guest Name' is equal an employee or some other entity completing the booking for the customer. In short, these particular observations and asscociated columns do not belong in this dataset. 

In [132]:
# Set null values to 0 for all high null count columns
clean_data=data
clean_data[high_nullcount_columns]= data[high_nullcount_columns].fillna(0)
clean_data

,Guest Name,Status,Arrive,Nts,Company,Made,Type,Rate,Disc,Override,PKG,Grp ID,Clerk,Conf #,Unnamed: 14
0,a6adf0970ec595255cb5417a5dfe6ce0f0c4608ed93dc0...,CXL,01/01/21,1,FRATERNAL OR,12/31/20,SK,119,0,0,0,0,ALeRoy,84406,0
1,a9d6016b3827799118c1f1feea23d00068b8cf79d9b40e...,OUT,01/01/21,1,0,12/31/20,SQQ,116.1,AAA,0,0,0,Tdahmer,84416,0
2,70907cf1d1f5659b43c5fb616417a0b87975cb93e0abe9...,OUT,01/01/21,1,0,01/01/21,SK,119,0,0,0,0,ALeRoy,84422,0
3,c098f7f6aab9e1288456ee87e57a19c100d7d6f861e0ff...,CXL,01/01/21,1,0,01/01/21,SQQ,109.65,MR,0,0,0,Lobrigo,84431,0
4,f112a154cdc31bfd940fec55423f8ed38e4018642fade4...,OUT,01/01/21,2,0,12/28/20,SQQ,98.1,BW,98.1,0,0,Admin,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51722,05b6d87c80678981de63ef2b3472c6b72da1b4b9ca703f...,GTD,05/07/24,1,0,02/13/24,SK,119,BC9,93.18,0,0,Admin,625258271-01,0
51723,8da1d8cff528fbc332a062146526ad2e522097a0734b7d...,CXL,05/07/24,1,(EPS) EXPEDI,04/16/24,SKA,95.46,EC9,95.4,0,0,Admin,860986212-01,0
51724,a2fc3ad7736a52b9aaa7ea983d15b30ddfd0d4ab1aa409...,GTD,05/07/24,2,0,04/19/24,SQQ,120.84,ECR,121.64,0,0,Admin,209536113-01,0
51725,4745091e4e1a796c1375584cabee8d66ae606655730d74...,GTD,05/07/24,2,0,04/17/24,SQQ,120.93,BW,120.93,0,0,Admin,776126515-01,0


In [133]:
# Now I will drop all the unwanted rows and columns from the dataset

guest_only_data = data
guest_only_data.drop(columns=['PKG', 'Grp ID', 'Unnamed: 14'], inplace=True)
guest_only_data

,Guest Name,Status,Arrive,Nts,Company,Made,Type,Rate,Disc,Override,Clerk,Conf #
0,a6adf0970ec595255cb5417a5dfe6ce0f0c4608ed93dc0...,CXL,01/01/21,1,FRATERNAL OR,12/31/20,SK,119,0,0,ALeRoy,84406
1,a9d6016b3827799118c1f1feea23d00068b8cf79d9b40e...,OUT,01/01/21,1,0,12/31/20,SQQ,116.1,AAA,0,Tdahmer,84416
2,70907cf1d1f5659b43c5fb616417a0b87975cb93e0abe9...,OUT,01/01/21,1,0,01/01/21,SK,119,0,0,ALeRoy,84422
3,c098f7f6aab9e1288456ee87e57a19c100d7d6f861e0ff...,CXL,01/01/21,1,0,01/01/21,SQQ,109.65,MR,0,Lobrigo,84431
4,f112a154cdc31bfd940fec55423f8ed38e4018642fade4...,OUT,01/01/21,2,0,12/28/20,SQQ,98.1,BW,98.1,Admin,0
...,...,...,...,...,...,...,...,...,...,...,...,...
51722,05b6d87c80678981de63ef2b3472c6b72da1b4b9ca703f...,GTD,05/07/24,1,0,02/13/24,SK,119,BC9,93.18,Admin,625258271-01
51723,8da1d8cff528fbc332a062146526ad2e522097a0734b7d...,CXL,05/07/24,1,(EPS) EXPEDI,04/16/24,SKA,95.46,EC9,95.4,Admin,860986212-01
51724,a2fc3ad7736a52b9aaa7ea983d15b30ddfd0d4ab1aa409...,GTD,05/07/24,2,0,04/19/24,SQQ,120.84,ECR,121.64,Admin,209536113-01
51725,4745091e4e1a796c1375584cabee8d66ae606655730d74...,GTD,05/07/24,2,0,04/17/24,SQQ,120.93,BW,120.93,Admin,776126515-01


## Side Note:
### The data in the 'Guest Name' column in booking_server_df  is also reflected in the 'Clerk' columns. This could be represented as a 1 to many foreign key relationship in an SQL database schema.



In [134]:
### Now Lets analyze our new, cleaner dataset. All of the dtypes are still just 'object'so df.describe is useless for now.
guest_only_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51727 entries, 0 to 51726
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Guest Name  51727 non-null  object
 1   Status      51727 non-null  object
 2   Arrive      51727 non-null  object
 3   Nts         51727 non-null  object
 4   Company     51727 non-null  object
 5   Made        51727 non-null  object
 6   Type        51727 non-null  object
 7   Rate        51727 non-null  object
 8   Disc        51727 non-null  object
 9   Override    51727 non-null  object
 10  Clerk       51727 non-null  object
 11  Conf #      51727 non-null  object
dtypes: object(12)
memory usage: 4.7+ MB


In [135]:
# Now that we have gotten rid of the null values lets eyeball the data again
guest_only_data

,Guest Name,Status,Arrive,Nts,Company,Made,Type,Rate,Disc,Override,Clerk,Conf #
0,a6adf0970ec595255cb5417a5dfe6ce0f0c4608ed93dc0...,CXL,01/01/21,1,FRATERNAL OR,12/31/20,SK,119,0,0,ALeRoy,84406
1,a9d6016b3827799118c1f1feea23d00068b8cf79d9b40e...,OUT,01/01/21,1,0,12/31/20,SQQ,116.1,AAA,0,Tdahmer,84416
2,70907cf1d1f5659b43c5fb616417a0b87975cb93e0abe9...,OUT,01/01/21,1,0,01/01/21,SK,119,0,0,ALeRoy,84422
3,c098f7f6aab9e1288456ee87e57a19c100d7d6f861e0ff...,CXL,01/01/21,1,0,01/01/21,SQQ,109.65,MR,0,Lobrigo,84431
4,f112a154cdc31bfd940fec55423f8ed38e4018642fade4...,OUT,01/01/21,2,0,12/28/20,SQQ,98.1,BW,98.1,Admin,0
...,...,...,...,...,...,...,...,...,...,...,...,...
51722,05b6d87c80678981de63ef2b3472c6b72da1b4b9ca703f...,GTD,05/07/24,1,0,02/13/24,SK,119,BC9,93.18,Admin,625258271-01
51723,8da1d8cff528fbc332a062146526ad2e522097a0734b7d...,CXL,05/07/24,1,(EPS) EXPEDI,04/16/24,SKA,95.46,EC9,95.4,Admin,860986212-01
51724,a2fc3ad7736a52b9aaa7ea983d15b30ddfd0d4ab1aa409...,GTD,05/07/24,2,0,04/19/24,SQQ,120.84,ECR,121.64,Admin,209536113-01
51725,4745091e4e1a796c1375584cabee8d66ae606655730d74...,GTD,05/07/24,2,0,04/17/24,SQQ,120.93,BW,120.93,Admin,776126515-01


In [136]:
# It looks like a few of these columns should now convert nicely to different dtypes now
# I want to convert the 'Made' and 'Arrive' columns to pandas datetime objects.
# I want convert the 'Nts' (Nights), to pandas int64 objects
# I want convert the 'Rate', and 'Override' columns to pandas float64 objects

guest_only_data = guest_only_data.copy() # This line prevents the 'SettingCopyWithWarning' output

guest_only_data['Made'] = pd.to_datetime(guest_only_data['Made'], format='%m/%d/%y')
guest_only_data['Arrive'] = pd.to_datetime(guest_only_data[ 'Arrive'], format='%m/%d/%y')

guest_only_data = guest_only_data.copy()
guest_only_data['Nts'] = pd.to_numeric(guest_only_data['Nts'], errors='coerce')

guest_only_data = guest_only_data.copy()

guest_only_data['Rate'] = pd.to_numeric(guest_only_data['Rate'], errors='coerce')
guest_only_data['Override'] = pd.to_numeric(guest_only_data['Override'], errors='coerce')
guest_only_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51727 entries, 0 to 51726
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Guest Name  51727 non-null  object        
 1   Status      51727 non-null  object        
 2   Arrive      51727 non-null  datetime64[ns]
 3   Nts         51727 non-null  int64         
 4   Company     51727 non-null  object        
 5   Made        51727 non-null  datetime64[ns]
 6   Type        51727 non-null  object        
 7   Rate        51727 non-null  float64       
 8   Disc        51727 non-null  object        
 9   Override    51727 non-null  float64       
 10  Clerk       51727 non-null  object        
 11  Conf #      51727 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(7)
memory usage: 4.7+ MB


### I think the rest of the dtypes are fine for now. The dataframe looks much more digestable. Now I'm going do some summary statistic analysis on some of the key columns. In a machine learning model, the dependent Y variable will be the 'Status' feature. Let's see what the values consist of.

In [137]:
guest_only_data.Status.unique()

array(['CXL', 'OUT', 'GTD', 'NS', 'HLD', 'IN'], dtype=object)

In [138]:
# The only values that appear in 'Status' Column are 'CXL' (Canceled) and 'OUT' (Checked OUT), perfect boolean values

guest_only_data.Status = guest_only_data.Status.map(lambda x: 1 if x == 'CXL' else 0)
guest_only_data.Status = guest_only_data['Status'].astype(bool)
guest_only_data.Status = guest_only_data.Status.map({True:1, False:0})
guest_only_data.rename(columns={'Status':'is_canceled','Arrive':'scheduled_arrival','Nts':'num_of_nights', 'Made':'date_booking_made', 
                                'Disc':'Discount'}, inplace=True)
guest_only_data = guest_only_data.copy()

guest_only_data

,Guest Name,is_canceled,scheduled_arrival,num_of_nights,Company,date_booking_made,Type,Rate,Discount,Override,Clerk,Conf #
0,a6adf0970ec595255cb5417a5dfe6ce0f0c4608ed93dc0...,1,2021-01-01,1,FRATERNAL OR,2020-12-31,SK,119.00,0,0.00,ALeRoy,84406
1,a9d6016b3827799118c1f1feea23d00068b8cf79d9b40e...,0,2021-01-01,1,0,2020-12-31,SQQ,116.10,AAA,0.00,Tdahmer,84416
2,70907cf1d1f5659b43c5fb616417a0b87975cb93e0abe9...,0,2021-01-01,1,0,2021-01-01,SK,119.00,0,0.00,ALeRoy,84422
3,c098f7f6aab9e1288456ee87e57a19c100d7d6f861e0ff...,1,2021-01-01,1,0,2021-01-01,SQQ,109.65,MR,0.00,Lobrigo,84431
4,f112a154cdc31bfd940fec55423f8ed38e4018642fade4...,0,2021-01-01,2,0,2020-12-28,SQQ,98.10,BW,98.10,Admin,0
...,...,...,...,...,...,...,...,...,...,...,...,...
51722,05b6d87c80678981de63ef2b3472c6b72da1b4b9ca703f...,0,2024-05-07,1,0,2024-02-13,SK,119.00,BC9,93.18,Admin,625258271-01
51723,8da1d8cff528fbc332a062146526ad2e522097a0734b7d...,1,2024-05-07,1,(EPS) EXPEDI,2024-04-16,SKA,95.46,EC9,95.40,Admin,860986212-01
51724,a2fc3ad7736a52b9aaa7ea983d15b30ddfd0d4ab1aa409...,0,2024-05-07,2,0,2024-04-19,SQQ,120.84,ECR,121.64,Admin,209536113-01
51725,4745091e4e1a796c1375584cabee8d66ae606655730d74...,0,2024-05-07,2,0,2024-04-17,SQQ,120.93,BW,120.93,Admin,776126515-01


In [139]:
# Check data types
guest_only_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51727 entries, 0 to 51726
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Guest Name         51727 non-null  object        
 1   is_canceled        51727 non-null  int64         
 2   scheduled_arrival  51727 non-null  datetime64[ns]
 3   num_of_nights      51727 non-null  int64         
 4   Company            51727 non-null  object        
 5   date_booking_made  51727 non-null  datetime64[ns]
 6   Type               51727 non-null  object        
 7   Rate               51727 non-null  float64       
 8   Discount           51727 non-null  object        
 9   Override           51727 non-null  float64       
 10  Clerk              51727 non-null  object        
 11  Conf #             51727 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(2), object(6)
memory usage: 4.7+ MB


### It is possible to derive new useful features from the data already contained in the dataset. For example, it may be useful for our predictions to also know the day of the week, and the month of the shedule arrival dates and the dates the reservation was booked.
### It may also be useful for classification to know if the Guest is a repeated booker and if they have cancelled before.

In [140]:
guest_data_df = guest_only_data.copy() # just making a copy of our dataframe so far to be safe

guest_only_data = guest_only_data.copy() # prevents SettingCopyWithWarning message


In [141]:
# It will be useful to know whether or not a guest is a unique booker or not for classifications and regressions
# the value_counts Series method is invaluable

guest_counts = guest_only_data['Guest Name'].value_counts()

# to generate the list of names of repeat bookers a list comprehension is the cleanest approach
# this list comprehension exploits the reverse lookup quality of Series.value_counts() objects

duplicate_guest_names = [name for name in guest_only_data['Guest Name'] if guest_counts[name] > 1]


repeat_guest_df = guest_only_data[guest_only_data['Guest Name'].isin(duplicate_guest_names)]


repeat_guest = repeat_guest_df['Guest Name'].unique()

guest_only_data['is_repeated_guest'] = guest_only_data['Guest Name'].isin(duplicate_guest_names)

guest_only_data['is_repeated_guest']= guest_only_data['is_repeated_guest'].map({True:1, False:0})

guest_only_data

,Guest Name,is_canceled,scheduled_arrival,num_of_nights,Company,date_booking_made,Type,Rate,Discount,Override,Clerk,Conf #,is_repeated_guest
0,a6adf0970ec595255cb5417a5dfe6ce0f0c4608ed93dc0...,1,2021-01-01,1,FRATERNAL OR,2020-12-31,SK,119.00,0,0.00,ALeRoy,84406,1
1,a9d6016b3827799118c1f1feea23d00068b8cf79d9b40e...,0,2021-01-01,1,0,2020-12-31,SQQ,116.10,AAA,0.00,Tdahmer,84416,0
2,70907cf1d1f5659b43c5fb616417a0b87975cb93e0abe9...,0,2021-01-01,1,0,2021-01-01,SK,119.00,0,0.00,ALeRoy,84422,1
3,c098f7f6aab9e1288456ee87e57a19c100d7d6f861e0ff...,1,2021-01-01,1,0,2021-01-01,SQQ,109.65,MR,0.00,Lobrigo,84431,0
4,f112a154cdc31bfd940fec55423f8ed38e4018642fade4...,0,2021-01-01,2,0,2020-12-28,SQQ,98.10,BW,98.10,Admin,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51722,05b6d87c80678981de63ef2b3472c6b72da1b4b9ca703f...,0,2024-05-07,1,0,2024-02-13,SK,119.00,BC9,93.18,Admin,625258271-01,0
51723,8da1d8cff528fbc332a062146526ad2e522097a0734b7d...,1,2024-05-07,1,(EPS) EXPEDI,2024-04-16,SKA,95.46,EC9,95.40,Admin,860986212-01,0
51724,a2fc3ad7736a52b9aaa7ea983d15b30ddfd0d4ab1aa409...,0,2024-05-07,2,0,2024-04-19,SQQ,120.84,ECR,121.64,Admin,209536113-01,0
51725,4745091e4e1a796c1375584cabee8d66ae606655730d74...,0,2024-05-07,2,0,2024-04-17,SQQ,120.93,BW,120.93,Admin,776126515-01,1


In [142]:
repeat_guest_df = repeat_guest_df.sort_values(by='date_booking_made')
repeat_guest_df

,Guest Name,is_canceled,scheduled_arrival,num_of_nights,Company,date_booking_made,Type,Rate,Discount,Override,Clerk,Conf #
153,dce424c244435f948598dbb7381425d43bbffd68fffd3e...,0,2021-01-14,0,0,2020-02-27,SQQ,128.00,SR9,79.95,Admin,0
4299,1e30bf72f0e4eead0c2e7a16c0362d254da451c7de7871...,1,2021-02-10,28,0,2020-03-09,SK,229.00,0,109.00,emontoya,76286
4224,10f91fdfba6b24b59362b63a54431cf950a75ef2a2f5f9...,1,2021-02-05,3,0,2020-05-23,SK,109.65,BK9,112.23,Admin,0
4225,10f91fdfba6b24b59362b63a54431cf950a75ef2a2f5f9...,1,2021-02-05,3,GETAROOM.COM,2020-05-23,SK,129.00,15D,95.40,Admin,0
5230,8c11ea8cc7bbcdfc436602b6157f7fbd4fbef278d756a9...,1,2021-04-23,2,0,2020-06-30,SK,139.00,2UB,115.93,Admin,0
...,...,...,...,...,...,...,...,...,...,...,...,...
51547,ed26e497acd1720fb77c2e556f8cfad82a451fd78599e3...,0,2024-05-05,1,0,2024-05-05,SK,120.93,BW,120.93,Admin,163140616-01
51590,6999934942cad24320107b5cb9c548afb5a80195aaafb3...,0,2024-05-05,2,0,2024-05-05,SQQ,120.93,BW,120.93,Admin,361840415-01
51585,37a2b685da53825f779aa98d1eca64f77105754f60a004...,0,2024-05-05,1,0,2024-05-05,SK,125.10,FX2,90.28,Admin,200840610-01
51568,64bcc44167f5c0d599030cb844e97aafbfa800bf4acd78...,0,2024-05-05,1,0,2024-05-05,SK,149.00,BC1,134.10,Admin,859740711-01


In [143]:
repeat_guest_df = repeat_guest_df.sort_values(by=['Guest Name', 'date_booking_made'])

In [144]:
guest_only_data

,Guest Name,is_canceled,scheduled_arrival,num_of_nights,Company,date_booking_made,Type,Rate,Discount,Override,Clerk,Conf #,is_repeated_guest
0,a6adf0970ec595255cb5417a5dfe6ce0f0c4608ed93dc0...,1,2021-01-01,1,FRATERNAL OR,2020-12-31,SK,119.00,0,0.00,ALeRoy,84406,1
1,a9d6016b3827799118c1f1feea23d00068b8cf79d9b40e...,0,2021-01-01,1,0,2020-12-31,SQQ,116.10,AAA,0.00,Tdahmer,84416,0
2,70907cf1d1f5659b43c5fb616417a0b87975cb93e0abe9...,0,2021-01-01,1,0,2021-01-01,SK,119.00,0,0.00,ALeRoy,84422,1
3,c098f7f6aab9e1288456ee87e57a19c100d7d6f861e0ff...,1,2021-01-01,1,0,2021-01-01,SQQ,109.65,MR,0.00,Lobrigo,84431,0
4,f112a154cdc31bfd940fec55423f8ed38e4018642fade4...,0,2021-01-01,2,0,2020-12-28,SQQ,98.10,BW,98.10,Admin,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51722,05b6d87c80678981de63ef2b3472c6b72da1b4b9ca703f...,0,2024-05-07,1,0,2024-02-13,SK,119.00,BC9,93.18,Admin,625258271-01,0
51723,8da1d8cff528fbc332a062146526ad2e522097a0734b7d...,1,2024-05-07,1,(EPS) EXPEDI,2024-04-16,SKA,95.46,EC9,95.40,Admin,860986212-01,0
51724,a2fc3ad7736a52b9aaa7ea983d15b30ddfd0d4ab1aa409...,0,2024-05-07,2,0,2024-04-19,SQQ,120.84,ECR,121.64,Admin,209536113-01,0
51725,4745091e4e1a796c1375584cabee8d66ae606655730d74...,0,2024-05-07,2,0,2024-04-17,SQQ,120.93,BW,120.93,Admin,776126515-01,1


In [145]:
# Get the number of all arrival dates
num_arrival_dates = guest_only_data.scheduled_arrival.shape[0]
guest_only_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51727 entries, 0 to 51726
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Guest Name         51727 non-null  object        
 1   is_canceled        51727 non-null  int64         
 2   scheduled_arrival  51727 non-null  datetime64[ns]
 3   num_of_nights      51727 non-null  int64         
 4   Company            51727 non-null  object        
 5   date_booking_made  51727 non-null  datetime64[ns]
 6   Type               51727 non-null  object        
 7   Rate               51727 non-null  float64       
 8   Discount           51727 non-null  object        
 9   Override           51727 non-null  float64       
 10  Clerk              51727 non-null  object        
 11  Conf #             51727 non-null  object        
 12  is_repeated_guest  51727 non-null  int64         
dtypes: datetime64[ns](2), float64(2), int64(3), object(6)
memory 

In [146]:
# Assuming 'date_booking_made' and 'scheduled_arrival' are datetime columns in your DataFrame
# Calculate the time difference and store it in a new column called 'booking_to_arrival_duration'
guest_only_data['booking_to_arrival_duration'] = guest_only_data['scheduled_arrival'] - guest_only_data['date_booking_made']

# Now guest_only_data contains a new column 'booking_to_arrival_duration' with the time difference
# Convert timedelta values to integers representing the number of days
guest_only_data['booking_to_arrival_duration'] = guest_only_data['booking_to_arrival_duration'].dt.days

# Now guest_only_data['booking_to_arrival_duration'] contains integer values representing the number of days
guest_only_data

,Guest Name,is_canceled,scheduled_arrival,num_of_nights,Company,date_booking_made,Type,Rate,Discount,Override,Clerk,Conf #,is_repeated_guest,booking_to_arrival_duration
0,a6adf0970ec595255cb5417a5dfe6ce0f0c4608ed93dc0...,1,2021-01-01,1,FRATERNAL OR,2020-12-31,SK,119.00,0,0.00,ALeRoy,84406,1,1
1,a9d6016b3827799118c1f1feea23d00068b8cf79d9b40e...,0,2021-01-01,1,0,2020-12-31,SQQ,116.10,AAA,0.00,Tdahmer,84416,0,1
2,70907cf1d1f5659b43c5fb616417a0b87975cb93e0abe9...,0,2021-01-01,1,0,2021-01-01,SK,119.00,0,0.00,ALeRoy,84422,1,0
3,c098f7f6aab9e1288456ee87e57a19c100d7d6f861e0ff...,1,2021-01-01,1,0,2021-01-01,SQQ,109.65,MR,0.00,Lobrigo,84431,0,0
4,f112a154cdc31bfd940fec55423f8ed38e4018642fade4...,0,2021-01-01,2,0,2020-12-28,SQQ,98.10,BW,98.10,Admin,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51722,05b6d87c80678981de63ef2b3472c6b72da1b4b9ca703f...,0,2024-05-07,1,0,2024-02-13,SK,119.00,BC9,93.18,Admin,625258271-01,0,84
51723,8da1d8cff528fbc332a062146526ad2e522097a0734b7d...,1,2024-05-07,1,(EPS) EXPEDI,2024-04-16,SKA,95.46,EC9,95.40,Admin,860986212-01,0,21
51724,a2fc3ad7736a52b9aaa7ea983d15b30ddfd0d4ab1aa409...,0,2024-05-07,2,0,2024-04-19,SQQ,120.84,ECR,121.64,Admin,209536113-01,0,18
51725,4745091e4e1a796c1375584cabee8d66ae606655730d74...,0,2024-05-07,2,0,2024-04-17,SQQ,120.93,BW,120.93,Admin,776126515-01,1,20


In [147]:
# Create month_of_arrival column
arrival_months = []
for i in range(num_arrival_dates):
    # Use iloc to access the scheduled_arrival by position
    arrival_months.append(guest_only_data['scheduled_arrival'].iloc[i].month)
guest_only_data['month_of_arrival'] = pd.Series(arrival_months, index=guest_only_data.index)



In [148]:
# Create day_of_arrival column

arrival_days = []
for i in range(num_arrival_dates):
    # Use iloc to access the scheduled_arrival by position
    arrival_days.append(guest_only_data['scheduled_arrival'].iloc[i].day)

# Creating the day_of_arrival Series directly with the DataFrame's index
guest_only_data['day_of_arrival'] = pd.Series(arrival_days, index=guest_only_data.index)


In [149]:
# Create day_of_booking column
booking_days = []
for i in range(num_arrival_dates):
    # Use iloc to access date_booking_made by position
    booking_days.append(guest_only_data['date_booking_made'].iloc[i].day)

# Creating the day_of_booking Series directly with the DataFrame's index
guest_only_data['day_of_booking'] = pd.Series(booking_days, index=guest_only_data.index)

In [150]:
arrival_weekdays = []
for i in range(num_arrival_dates):
    # Use iloc to access scheduled_arrival by position
    arrival_weekdays.append(guest_only_data['scheduled_arrival'].iloc[i].weekday())

# Creating the weekday_of_arrival Series directly with the DataFrame's index
guest_only_data['weekday_of_arrival'] = pd.Series(arrival_weekdays, index=guest_only_data.index)


In [151]:
booking_weekdays = []
for i in range(num_arrival_dates):
    # Use iloc to access date_booking_made by position
    booking_weekdays.append(guest_only_data['date_booking_made'].iloc[i].weekday())

# Creating the weekday_of_booking Series directly with the DataFrame's index
guest_only_data['weekday_of_booking'] = pd.Series(booking_weekdays, index=guest_only_data.index)

In [152]:
booking_months = []
for i in range(num_arrival_dates):
    # Use iloc to access date_booking_made by position
    booking_months.append(guest_only_data['date_booking_made'].iloc[i].weekday())

# Creating the weekday_of_booking Series directly with the DataFrame's index
guest_only_data['month_of_booking'] = pd.Series(booking_months, index=guest_only_data.index)

In [153]:
df_list = []
for guest_name in repeat_guest:
    filtered_df = repeat_guest_df[(repeat_guest_df['Guest Name'] == guest_name) & (repeat_guest_df['is_canceled'] == 0)]

    # Reset the counter and list for each guest
    previous_stays_list = [0] * len(filtered_df)
    num_of_previous_stays = 0

    # Iterate over the DataFrame rows
    for idx in range(1, len(filtered_df)):
        num_of_previous_stays += 1  # Increment counter for previous cancellations
        previous_stays_list[idx] = num_of_previous_stays

    # Assign the series after the loop
    filtered_df['previous_stays'] = pd.Series(previous_stays_list,dtype=int, index=filtered_df.index)
    df_list.append(filtered_df)



updated_df = pd.concat(df_list)
guest_only_data= pd.concat([guest_only_data, updated_df['previous_stays']], axis=1)

guest_only_data.fillna(0, inplace=True)
guest_only_data['previous_stays'] = guest_only_data['previous_stays'].astype(int)
guest_only_data= guest_only_data.sort_values(by=['Guest Name', 'date_booking_made'])

guest_only_data

,Guest Name,is_canceled,scheduled_arrival,num_of_nights,Company,date_booking_made,Type,Rate,Discount,Override,Clerk,Conf #,is_repeated_guest,booking_to_arrival_duration,month_of_arrival,day_of_arrival,day_of_booking,weekday_of_arrival,weekday_of_booking,month_of_booking,previous_stays
21550,00023b76478a51c3ede5019cf5b88e7a70fbdb17449b86...,0,2022-06-09,4,0,2022-04-26,SQQ,97.01,BK5,97.01,Admin,0,0,44,6,9,26,3,1,1,0
32408,0004374957a95fa2436fa8053474f8b466eefd81205643...,0,2023-02-04,1,0,2022-12-21,SQQ,279.00,EX1,279.00,Admin,0,0,45,2,4,21,5,2,2,0
21486,0005f861dcdb4b1c14daa18cad0081c59059f5eb968c15...,0,2022-06-07,1,LOTS OF HOTE,2022-06-06,SK,90.40,LP2,90.40,Admin,0,0,1,6,7,6,1,0,0,0
11061,0006cdbe73d05775693b451356f7ee8e8bcd7ef31e06ad...,0,2022-11-03,2,0,2022-10-05,SQQ,139.00,EX1,139.00,Admin,0,0,29,11,3,5,3,2,2,0
7958,00081fcfeec47545a100b55d0b7fb743454fbcd2b261d1...,0,2022-01-13,3,0,2021-12-27,SQQ,179.00,BK1,179.00,Admin,0,0,17,1,13,27,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30189,fffc46acaf12480788bd5b87056a346688c12a38f57391...,1,2023-11-11,1,0,2023-11-03,SQQ,169.00,BC1,169.00,Admin,656724170-01,1,8,11,11,3,5,4,4,0
30190,fffc46acaf12480788bd5b87056a346688c12a38f57391...,0,2023-11-11,1,0,2023-11-07,SDD,169.00,BC1,169.00,Admin,699554675-01,1,4,11,11,7,5,1,1,0
7890,ffff94c2eddde6bc97c57e130d05c904c5c4fcf2fc592a...,0,2022-01-12,1,0,2021-12-13,SQQ,97.30,EC9,100.43,Admin,0,0,30,1,12,13,2,0,0,0
1533,ffffb29e6e5863e74a77550ad998aecf1c7ef35848a147...,0,2021-11-01,2,0,2021-11-01,SK,108.90,BW,0.00,Lobrigo,94936,1,0,11,1,1,0,0,0,0


In [154]:

df_list = []
for guest_name in repeat_guest:
    filtered_df = repeat_guest_df[(repeat_guest_df['Guest Name'] == guest_name) & (repeat_guest_df['is_canceled'] == 1)]

    # Reset the counter and list for each guest
    previous_cancels_list = [0] * len(filtered_df)
    num_of_previous_cancels = 0

    # Iterate over the DataFrame rows
    for idx in range(1, len(filtered_df)):
        num_of_previous_cancels += 1  # Increment counter for previous cancellations
        previous_cancels_list[idx] = num_of_previous_cancels

    # Assign the series after the loop with specified dtype
    filtered_df['previous_cancellations'] = pd.Series(previous_cancels_list, dtype=int, index=filtered_df.index)
    df_list.append(filtered_df)

# Concatenate all DataFrames collected in the list
updated_df = pd.concat(df_list)

# Assuming 'guest_only_data' is a DataFrame defined elsewhere and ready to be used here
guest_only_data = pd.concat([guest_only_data, updated_df[['previous_cancellations']]], axis=1)

# Fill NA values with zero and ensure integer type for the 'previous_cancellations' column
guest_only_data.fillna(0, inplace=True)
guest_only_data['previous_cancellations'] = guest_only_data['previous_cancellations'].astype(int)

# Sorting by 'Guest Name' and 'date_booking_made' assuming these columns exist
guest_only_data = guest_only_data.sort_values(by=['Guest Name', 'date_booking_made'])

# Display the DataFrame (optional, depending on your environment you might want to print or return it)
guest_only_data

,Guest Name,is_canceled,scheduled_arrival,num_of_nights,Company,date_booking_made,Type,Rate,Discount,Override,Clerk,Conf #,is_repeated_guest,booking_to_arrival_duration,month_of_arrival,day_of_arrival,day_of_booking,weekday_of_arrival,weekday_of_booking,month_of_booking,previous_stays,previous_cancellations
21550,00023b76478a51c3ede5019cf5b88e7a70fbdb17449b86...,0,2022-06-09,4,0,2022-04-26,SQQ,97.01,BK5,97.01,Admin,0,0,44,6,9,26,3,1,1,0,0
32408,0004374957a95fa2436fa8053474f8b466eefd81205643...,0,2023-02-04,1,0,2022-12-21,SQQ,279.00,EX1,279.00,Admin,0,0,45,2,4,21,5,2,2,0,0
21486,0005f861dcdb4b1c14daa18cad0081c59059f5eb968c15...,0,2022-06-07,1,LOTS OF HOTE,2022-06-06,SK,90.40,LP2,90.40,Admin,0,0,1,6,7,6,1,0,0,0,0
11061,0006cdbe73d05775693b451356f7ee8e8bcd7ef31e06ad...,0,2022-11-03,2,0,2022-10-05,SQQ,139.00,EX1,139.00,Admin,0,0,29,11,3,5,3,2,2,0,0
7958,00081fcfeec47545a100b55d0b7fb743454fbcd2b261d1...,0,2022-01-13,3,0,2021-12-27,SQQ,179.00,BK1,179.00,Admin,0,0,17,1,13,27,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30189,fffc46acaf12480788bd5b87056a346688c12a38f57391...,1,2023-11-11,1,0,2023-11-03,SQQ,169.00,BC1,169.00,Admin,656724170-01,1,8,11,11,3,5,4,4,0,0
30190,fffc46acaf12480788bd5b87056a346688c12a38f57391...,0,2023-11-11,1,0,2023-11-07,SDD,169.00,BC1,169.00,Admin,699554675-01,1,4,11,11,7,5,1,1,0,0
7890,ffff94c2eddde6bc97c57e130d05c904c5c4fcf2fc592a...,0,2022-01-12,1,0,2021-12-13,SQQ,97.30,EC9,100.43,Admin,0,0,30,1,12,13,2,0,0,0,0
1533,ffffb29e6e5863e74a77550ad998aecf1c7ef35848a147...,0,2021-11-01,2,0,2021-11-01,SK,108.90,BW,0.00,Lobrigo,94936,1,0,11,1,1,0,0,0,0,0


In [155]:
# Assuming 'date_booking_made' and 'scheduled_arrival' are datetime columns in your DataFrame
# Calculate the time difference and store it in a new column called 'booking_to_arrival_duration'
guest_only_data['booking_to_arrival_duration'] = guest_only_data['scheduled_arrival'] - guest_only_data['date_booking_made']

# Now guest_only_data contains a new column 'booking_to_arrival_duration' with the time difference
guest_only_data['booking_to_arrival_duration'] = guest_only_data['booking_to_arrival_duration'].dt.days

guest_only_data

,Guest Name,is_canceled,scheduled_arrival,num_of_nights,Company,date_booking_made,Type,Rate,Discount,Override,Clerk,Conf #,is_repeated_guest,booking_to_arrival_duration,month_of_arrival,day_of_arrival,day_of_booking,weekday_of_arrival,weekday_of_booking,month_of_booking,previous_stays,previous_cancellations
21550,00023b76478a51c3ede5019cf5b88e7a70fbdb17449b86...,0,2022-06-09,4,0,2022-04-26,SQQ,97.01,BK5,97.01,Admin,0,0,44,6,9,26,3,1,1,0,0
32408,0004374957a95fa2436fa8053474f8b466eefd81205643...,0,2023-02-04,1,0,2022-12-21,SQQ,279.00,EX1,279.00,Admin,0,0,45,2,4,21,5,2,2,0,0
21486,0005f861dcdb4b1c14daa18cad0081c59059f5eb968c15...,0,2022-06-07,1,LOTS OF HOTE,2022-06-06,SK,90.40,LP2,90.40,Admin,0,0,1,6,7,6,1,0,0,0,0
11061,0006cdbe73d05775693b451356f7ee8e8bcd7ef31e06ad...,0,2022-11-03,2,0,2022-10-05,SQQ,139.00,EX1,139.00,Admin,0,0,29,11,3,5,3,2,2,0,0
7958,00081fcfeec47545a100b55d0b7fb743454fbcd2b261d1...,0,2022-01-13,3,0,2021-12-27,SQQ,179.00,BK1,179.00,Admin,0,0,17,1,13,27,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30189,fffc46acaf12480788bd5b87056a346688c12a38f57391...,1,2023-11-11,1,0,2023-11-03,SQQ,169.00,BC1,169.00,Admin,656724170-01,1,8,11,11,3,5,4,4,0,0
30190,fffc46acaf12480788bd5b87056a346688c12a38f57391...,0,2023-11-11,1,0,2023-11-07,SDD,169.00,BC1,169.00,Admin,699554675-01,1,4,11,11,7,5,1,1,0,0
7890,ffff94c2eddde6bc97c57e130d05c904c5c4fcf2fc592a...,0,2022-01-12,1,0,2021-12-13,SQQ,97.30,EC9,100.43,Admin,0,0,30,1,12,13,2,0,0,0,0
1533,ffffb29e6e5863e74a77550ad998aecf1c7ef35848a147...,0,2021-11-01,2,0,2021-11-01,SK,108.90,BW,0.00,Lobrigo,94936,1,0,11,1,1,0,0,0,0,0


In [156]:
guest_only_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51727 entries, 21550 to 1573
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Guest Name                   51727 non-null  object        
 1   is_canceled                  51727 non-null  int64         
 2   scheduled_arrival            51727 non-null  datetime64[ns]
 3   num_of_nights                51727 non-null  int64         
 4   Company                      51727 non-null  object        
 5   date_booking_made            51727 non-null  datetime64[ns]
 6   Type                         51727 non-null  object        
 7   Rate                         51727 non-null  float64       
 8   Discount                     51727 non-null  object        
 9   Override                     51727 non-null  float64       
 10  Clerk                        51727 non-null  object        
 11  Conf #                       51727 non

In [157]:
guest_only_data.describe()

,is_canceled,num_of_nights,Rate,Override,is_repeated_guest,booking_to_arrival_duration,month_of_arrival,day_of_arrival,day_of_booking,weekday_of_arrival,weekday_of_booking,month_of_booking,previous_stays,previous_cancellations
count,51727.000000,51727.000000,51727.000000,51727.000000,51727.000000,51727.000000,51727.000000,51727.000000,51727.000000,51727.000000,51727.000000,51727.000000,51727.000000,51727.000000
mean,0.303497,2.021207,140.971328,95.450203,0.490904,27.583892,6.162314,15.614302,15.473795,2.992209,2.760377,2.760377,0.917103,115.474433
std,0.459772,2.046882,55.237107,69.999159,0.499922,43.969963,3.586405,8.725452,8.738066,1.925556,1.939227,1.939227,5.190812,429.370420
min,0.000000,0.000000,-16.800000,0.000000,0.000000,-3.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,107.100000,35.540000,0.000000,1.000000,3.000000,8.000000,8.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,0.000000,1.000000,129.000000,100.580000,0.000000,10.000000,6.000000,16.000000,15.000000,3.000000,3.000000,3.000000,0.000000,0.000000
75%,1.000000,2.000000,160.900000,135.150000,1.000000,36.000000,10.000000,23.000000,23.000000,5.000000,4.000000,4.000000,0.000000,0.000000
max,1.000000,86.000000,900.000000,759.000000,1.000000,740.000000,12.000000,31.000000,31.000000,6.000000,6.000000,6.000000,123.000000,2872.000000


### The mean of 'is_canceled' tells me that 30% of these observations were canceled bookings. With an extremely low standard deviation of 0.46 because the column values are all either 0's or 1's
### The mean 'of num_of_nights' is about 2 nights, with a fairly low standard deviation of 1.9
### The mean Rate is  141 with a standard deviation of about 55
### The mean of Override is about 95.5 with a relatively high standard deviation of 70.

### This dataset is sufficiently preprocessed and ready for further testing.  Next I will analyze the features using a correlation matrix, but before that, I have to make sure all of the features are numerical.

In [158]:
guest_only_data

,Guest Name,is_canceled,scheduled_arrival,num_of_nights,Company,date_booking_made,Type,Rate,Discount,Override,Clerk,Conf #,is_repeated_guest,booking_to_arrival_duration,month_of_arrival,day_of_arrival,day_of_booking,weekday_of_arrival,weekday_of_booking,month_of_booking,previous_stays,previous_cancellations
21550,00023b76478a51c3ede5019cf5b88e7a70fbdb17449b86...,0,2022-06-09,4,0,2022-04-26,SQQ,97.01,BK5,97.01,Admin,0,0,44,6,9,26,3,1,1,0,0
32408,0004374957a95fa2436fa8053474f8b466eefd81205643...,0,2023-02-04,1,0,2022-12-21,SQQ,279.00,EX1,279.00,Admin,0,0,45,2,4,21,5,2,2,0,0
21486,0005f861dcdb4b1c14daa18cad0081c59059f5eb968c15...,0,2022-06-07,1,LOTS OF HOTE,2022-06-06,SK,90.40,LP2,90.40,Admin,0,0,1,6,7,6,1,0,0,0,0
11061,0006cdbe73d05775693b451356f7ee8e8bcd7ef31e06ad...,0,2022-11-03,2,0,2022-10-05,SQQ,139.00,EX1,139.00,Admin,0,0,29,11,3,5,3,2,2,0,0
7958,00081fcfeec47545a100b55d0b7fb743454fbcd2b261d1...,0,2022-01-13,3,0,2021-12-27,SQQ,179.00,BK1,179.00,Admin,0,0,17,1,13,27,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30189,fffc46acaf12480788bd5b87056a346688c12a38f57391...,1,2023-11-11,1,0,2023-11-03,SQQ,169.00,BC1,169.00,Admin,656724170-01,1,8,11,11,3,5,4,4,0,0
30190,fffc46acaf12480788bd5b87056a346688c12a38f57391...,0,2023-11-11,1,0,2023-11-07,SDD,169.00,BC1,169.00,Admin,699554675-01,1,4,11,11,7,5,1,1,0,0
7890,ffff94c2eddde6bc97c57e130d05c904c5c4fcf2fc592a...,0,2022-01-12,1,0,2021-12-13,SQQ,97.30,EC9,100.43,Admin,0,0,30,1,12,13,2,0,0,0,0
1533,ffffb29e6e5863e74a77550ad998aecf1c7ef35848a147...,0,2021-11-01,2,0,2021-11-01,SK,108.90,BW,0.00,Lobrigo,94936,1,0,11,1,1,0,0,0,0,0


In [159]:
columns_reordered =['Guest Name', 'date_booking_made','month_of_booking','day_of_booking', 'weekday_of_booking',
                    'scheduled_arrival','month_of_arrival','day_of_arrival','weekday_of_arrival', 
 'num_of_nights','is_repeated_guest', 'previous_stays','previous_cancellations', 'booking_to_arrival_duration', 'Rate', 'Discount','Override','Company','Type','Clerk','Conf #', 'is_canceled']

In [160]:
# final_df = guest_only_data[columns_reordered]
final_df = guest_only_data.copy()

In [161]:
columns_to_drop = ['Guest Name', 'Conf #', 'date_booking_made', 'scheduled_arrival', 
                               'Discount', 'Company', 'Clerk', 'Type', 'is_canceled']
final_df.drop(columns=columns_to_drop, inplace=True)

final_df

,num_of_nights,Rate,Override,is_repeated_guest,booking_to_arrival_duration,month_of_arrival,day_of_arrival,day_of_booking,weekday_of_arrival,weekday_of_booking,month_of_booking,previous_stays,previous_cancellations
21550,4,97.01,97.01,0,44,6,9,26,3,1,1,0,0
32408,1,279.00,279.00,0,45,2,4,21,5,2,2,0,0
21486,1,90.40,90.40,0,1,6,7,6,1,0,0,0,0
11061,2,139.00,139.00,0,29,11,3,5,3,2,2,0,0
7958,3,179.00,179.00,0,17,1,13,27,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30189,1,169.00,169.00,1,8,11,11,3,5,4,4,0,0
30190,1,169.00,169.00,1,4,11,11,7,5,1,1,0,0
7890,1,97.30,100.43,0,30,1,12,13,2,0,0,0,0
1533,2,108.90,0.00,1,0,11,1,1,0,0,0,0,0


In [55]:
final_df.to_csv('../My Data Science Projects/BW Reservations/BW_bookings_2021-present.csv')